## QA Test test

In [66]:
from collections import Counter
import nlptk
from nlptk import DictVsText
from gliner import GLiNER
from pathlib import Path
import os
from tabulate import tabulate
import json
import re
import copy
import time
from dataclasses import dataclass
import base64
import json
import tqdm
from dataclasses import dataclass
from pathlib import Path
import pandas as pd
import random
from nlptk import FileReader
from gliner import GLiNER
import phonenumbers
from phonenumbers import PhoneNumberFormat
import requests

PORT = "8001"
HOST = "10.0.0.105"
BASE_URL = f"http://{HOST}:{PORT}"
# BASE_URL = f"http://budgie.local:{PORT}"


@dataclass
class Endpoint:
    hello = f"{BASE_URL}/hello"
    static = f"{BASE_URL}/static"
    parse_resume_text = f"{BASE_URL}/parse_resume"
    parse_resume_doc = f"{BASE_URL}/parse_resume_doc"

home = Path.home()
data_dir = home.joinpath("Data/Jobscan/Resumes/QA_Resumes/May-2025")

paths  = [data_dir.joinpath(x) for x in os.listdir(data_dir) if re.search("(.docx|.pdf|.txt)$", x, re.IGNORECASE)]

ignore = ["Bullets + Hyphens Issue.docx", "EmilyKistnerResume copy.pdf", "Michelle Hytry Resume 2025.pdf" 
         "Project Development Manager_Sophie Tremblay.txt", "Resume_Waleska.docx",
         "Unicode issue_Patrick Jewell.pdf", "Web URL issue_Jennifer Barilla.pdf",
         "Zach Adams Resume GM.pdf"]
paths = [x for x in paths if not x.name in ignore]
paths.sort()
iter_paths = iter(paths)


country_codes = {
    "Qatar": "+974",
    "Japan": "+81",
    "South Korea": "+82",
    "Bulgaria": "+359"
}

In [49]:
def load_jsonlines(path):
    return [json.loads(x) for x in open(path)]

def test_parse_resume_doc(path: Path):
    filename = path.name
    with open(path, "rb") as fo:
        encoded_string = base64.b64encode(fo.read())

    payload = {"filename": filename, "filedata": encoded_string}
    response = requests.post(url=Endpoint.parse_resume_doc, data=payload)
    return response

def test_parse_resume_txt(text):
    data = {"text": text}
    headers = {"Content-Type": "application/json"}
    response = requests.post(Endpoint.parse_resume_text, headers=headers, data=json.dumps(data))
    return response

def run(path, problem):
    response = test_parse_resume_doc(path)
    basics = response.json()["jsonresume"]["basics"]
    text = response.json()["text"]
    lines = [x for x in text.split("\n") if x.strip()][:10]
    top_text = "\n".join(lines)
    print(path.name)
    print(problem)
    print("----------")
    print(json.dumps(basics, indent=2))
    print("\n ---------- \n")
    print(top_text)
    return top_text

nermodel = GLiNER.from_pretrained("urchade/gliner_medium-v2.1")



def _guess_country(tel):
    country_codes = {"Qatar": "+974", "Japan": "+81", "South Korea": "+82", "Bulgaria": "+359"}
    code_to_country = {"+974": "QA", "+81": "JP", "+82": "KR", "+359": "BG"}
    """
        "(+98)" -> Iran
    """
    code = tel.split()[0]
    
    if code in code_to_country:
        return code_to_country[code]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [50]:
problems = {
    "Brand Marketing Rep - Lucas Anderson.docx": ["job title"],
    "Bullets + Hyphens Issue.docx": [],
    "Emily Zhang.txt": ["job title", "website"],
    "EmilyKistnerResume copy.pdf": [],
    "Financial Services Exec_Kevin.docx": ["job title"],
    "Head of Marketing_Sofia Andersen.txt": ["job title", "phone", "website"],
    "Michelle Hytry Resume 2025.pdf": [],
    "Mojo Oloruntobi.pdf": ["job title"],
    "Name Issue_Elena Kim - Product Manager.pdf": ["phone"], #8 - Korea
    "Name issue_Sora Yamamoto - Systems Analyst.pdf": ["phone"], #9 - Japan
    "Project Development Manager_Sophie Tremblay.txt": [],
    "Resume_Waleska.docx": [],
    "Tariq El - Renewable Energy Engineer.docx": ["phone"], #12 - Qatar
    "URLs issue_Effat Zaferani.pdf": ["linkedin", "website"],
    "Unicode issue_Patrick Jewell.pdf": [],
    "Unicode_Eleanor Annese.docx": ["job title", "linkedin"],
    "Unicode_Travis Glenn.pdf": ["name"],
    "Web URL issue_Jennifer Barilla.pdf": [],
    "Zach Adams Resume GM.pdf": [],
    "Zara Nikolov - Sustainability Consultant.pdf": ["phone"] #19 - Bulgaria
}

In [75]:
[p.name for p in paths]

['Brand Marketing Rep - Lucas Anderson.docx',
 'Emily Zhang.txt',
 'Financial Services Exec_Kevin.docx',
 'Head of Marketing_Sofia Andersen.txt',
 'Michelle Hytry Resume 2025.pdf',
 'Mojo Oloruntobi.pdf',
 'Name Issue_Elena Kim - Product Manager.pdf',
 'Name issue_Sora Yamamoto - Systems Analyst.pdf',
 'Project Development Manager_Sophie Tremblay.txt',
 'Tariq El - Renewable Energy Engineer.docx',
 'URLs issue_Effat Zaferani.pdf',
 'Unicode_Eleanor Annese.docx',
 'Unicode_Travis Glenn.pdf',
 'Zara Nikolov - Sustainability Consultant.pdf']

In [88]:
# path = next(iter_paths)
path = paths[-1]
top_text = run(path, problems[path.name])

Zara Nikolov - Sustainability Consultant.pdf
['phone']
----------
{
  "name": "Zara Nikolov",
  "label": "Sustainability Consultant",
  "email": "zara.nikolov@email.com",
  "website": "",
  "phone": "088 924 1573",
  "url": "www.linkedin.com/in/zara-nikolov",
  "summary": "",
  "location": {},
  "profiles": [
    {
      "url": "",
      "network": "Linkedin",
      "username": ""
    },
    {
      "url": "",
      "network": "Github",
      "username": ""
    },
    {
      "url": "",
      "network": "Github",
      "username": ""
    },
    {
      "url": "www.zaranikolov.com",
      "network": "",
      "username": ""
    },
    {
      "url": "www.medium.com/@zaranikolov",
      "network": "",
      "username": ""
    },
    {
      "url": "www.github.com/zaranikolov",
      "network": "",
      "username": ""
    }
  ]
}

 ---------- 

 Zara Nikolov
Sustainability Consultant
Pravda Street 14, Apt 7B ,Sofia, Bulgaria 1000 | zara.nikolov@email.com | +359 88 924 1573
LinkedIn: www.

In [89]:
labels = ["JobTitle"]
entities = nermodel.predict_entities(top_text, labels, threshold=0.57)
entities

[{'start': 14,
  'end': 39,
  'text': 'Sustainability Consultant',
  'label': 'JobTitle',
  'score': 0.7380045056343079},
 {'start': 718,
  'end': 744,
  'text': 'Environmental Data Analyst',
  'label': 'JobTitle',
  'score': 0.5920465588569641}]

In [90]:
m = re.search(r"[\+\(]?[1-9][0-9 .\-\(\)]{8,}[0-9]", top_text)
tel = m.group()
tel
             

'+359 88 924 1573'

In [91]:
_guess_country(tel)

'BG'

In [92]:
format = PhoneNumberFormat.INTERNATIONAL
parsed_number = phonenumbers.parse(tel, "BG")
parsed_number
normalized_number = phonenumbers.format_number(parsed_number, format)
normalized_number

'+359 88 924 1573'